In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML
import gseapy as gp
from gseapy import Msigdb
from gseapy import GSEA
from gseapy import dotplot
import warnings
warnings.filterwarnings('ignore')

In [ ]:
adata = sc.read('../Data/dataset_annotated.h5ad')
violin_plot = ['classification','Basal', 'LumA', 'LumB', 'Her2', 'Normal']
cluster='Her2-G2'
score='Her2'

In [ ]:
msig = Msigdb()
gmt = msig.get_gmt(category='h.all', dbver="2025.1.Hs")

def plot_bars(tmp, ax):
    col1_props = tmp['pam50 subtype'].value_counts(normalize=True)
    col2_props = tmp['nhg'].value_counts(normalize=True)
    proportions = pd.concat([col1_props, col2_props], axis=1, keys=['pam50 subtype', 'nhg']).fillna(0)
    proportions.T.plot(
        kind='bar',
        stacked=True,
        colormap='tab20',
        edgecolor='black',
        ax=ax
    )

    ax.legend()
    ax.tick_params(rotation=0)
    ax.set_ylabel('Proportion')
    ax.set_title('PAM50 & NHG proportions')
    return ax, proportions


def plot_violin(tmp,ax):
    ax=sns.violinplot(tmp,ax=ax)
    ax.tick_params(rotation=45)
    ax.set_title('PAM50 scoring')
    return ax

def pathways(cluster,ax):
    expr = sc.get.rank_genes_groups_df(adata, group=cluster)[['names','scores']]
    expr.columns = ['gene_name', 'score'] 
    pre_res = gp.prerank(
        rnk=expr,  # DataFrame or path to .rnk file
        gene_sets=gmt,  # Or 'KEGG_2021_Human', 'Reactome_2022', etc.
        permutation_num=1000,  # recommended ≥1000
        seed=42,
        processes=4  # parallelization
    )

    ax = dotplot(pre_res.res2d,
             column="FDR q-val",
             cmap=plt.cm.viridis,
             size=4, # adjust dot size
             cutoff=0.25, show_ring=False,ax=ax)
    return ax, pre_res.res2d


In [ ]:
Markdown(f"""
# Cluster {cluster}""")

In [ ]:
fig,axes= plt.subplots(1,2, figsize=(12,5))
sc.pl.umap(adata, color=score,ax=axes[0], show=False)
axes[0].set_title(f"{score} - score")
sc.pl.umap(adata, color='classification',ax=axes[1], show=False)
axes[1].set_title('Clusters')
plt.show()


Figure 1: UMAP Visualization of Breast Cancer Samples Colored by Her2 Score and Assigned Clusters.

This figure displays the dimensionality reduction of the breast cancer RNA-Seq samples (same cohort as previously analyzed) using Uniform Manifold Approximation and Projection (UMAP). Each point in the plots represents an individual sample.

- Left panel: The UMAP space is colored based on a calculated Her2 score for each sample. This score was derived from the expression levels of specific genes from the PAM50 panel known to have high centroid values for the Her2-enriched subtype. The color bar indicates the range of the score, where samples with higher Her2 scores are depicted in yellow/green, and samples with lower scores are in purple/blue.

- Right panel: The same UMAP space is shown, with samples colored according to their assigned cluster. These clusters were previously identified based on the analysis and represent different breast cancer subtypes combined with Nottingham Histologic Grade (G2 or G3): Basal-G3, Her2-G2, Her2-G3, LumA-G2, LumB-G3, and Normal-G2, as indicated by the legend.

Interpretation: The UMAP projection effectively separates the samples into distinct regions corresponding to the different molecular subtypes. The spatial distribution of samples exhibiting high Her2 scores (left panel) strongly overlaps with the regions occupied by the Her2-G2 and Her2-G3 clusters (right panel). This visual correspondence confirms that the calculated Her2 score effectively identifies samples with a strong Her2-enriched gene expression signature and validates that the clustering method successfully grouped these samples together based on this molecular characteristic, while also capturing potential histological grade differences within the Her2 subtype.

In [ ]:
vln = sc.get.obs_df(adata, keys=violin_plot)
vln=vln[vln['classification']==cluster]
prop=sc.get.obs_df(adata, keys=['classification','pam50 subtype','nhg'])
prop=prop[prop['classification']==cluster]


fig,axes=plt.subplots(1,2, figsize=(12,5))
axes[0]=plot_violin(vln,axes[0])
axes[1],prop=plot_bars(prop,axes[1])
plt.show()

In [ ]:
Markdown(f"""

### PAM50 scoring - {cluster}

{vln.describe().to_html()}

### PAM50 & NHG proportions {cluster}

{prop.to_html()}
""")

Figure 2: Molecular and Histological Characterization of the Her2-G2 Cluster.

This figure provides a detailed characterization of the samples assigned to the Her2-G2 cluster (n=61) based on PAM50 subtype scoring and clinical Nottingham Histological Grade (NHG).

- Left panel: Violin plots displaying the distribution of PAM50 centroid scores for the samples within the Her2-G2 cluster. Each violin shows the density distribution of how strongly these samples score against the average gene expression profile (centroid) of each of the five intrinsic PAM50 subtypes (Basal-like, Luminal A, Luminal B, Her2-enriched, and Normal-like). Embedded box plots indicate the median and interquartile range. The Y-axis represents the PAM50 score, reflecting the molecular similarity of the Her2-G2 samples to each respective subtype centroid.

- Right panel: Stacked bar plots illustrating the proportions of samples within the Her2-G2 cluster according to their PAM50 molecular subtype assignment (left bar) and their Nottingham Histological Grade (NHG; right bar), based on clinical metadata. The legend specifies the color assigned to each subtype and grade category. This plot visualizes the composition of this specific cluster.

Interpretation: The violin plots on the left show that samples within the Her2-G2 cluster exhibit the highest scores for the Her2 centroid, confirming their primary molecular characteristic. While the Her2 scores are highest, there are also notable distributions for the Normal and Luminal A centroids, suggesting some molecular heterogeneity or transitional states within this group. The stacked bar plots on the right confirm the cluster's classification: the majority (59.0%) are assigned the Her2 PAM50 subtype, with smaller but significant proportions of Normal (21.3%) and LumA (18.0%) subtypes also present. Clinically, the cluster is predominantly composed of samples with Histological Grade 2 (45.9%), with a substantial minority of Grade 3 (47.5%), justifying the "G2" label primarily reflecting the most frequent grade, though noting the presence of G3 cases is important (especially in contrast to a potential separate Her2-G3 cluster). This figure highlights the molecular profile and histological grade composition of the Her2-G2 cluster.

In [ ]:
fig,axes=plt.subplots(1,1, figsize=(12,5))
axes,res=pathways(cluster,axes)
axes.set_title('GSEA')
# axes[1],prop=plot_bars(prop,axes[1])
plt.show()

In [ ]:
Markdown(f"""

### GSEA - {cluster}

{res[res['FDR q-val']<0.05].iloc[:,:-1].to_html()}

""")




Figure 3: Gene Set Enrichment Analysis (GSEA) Identifies Key Biological Pathways Characterizing the Her2-G2 Cluster.

This bubble plot presents the results of Gene Set Enrichment Analysis (GSEA) comparing the gene expression profiles of samples within the Her2-G2 cluster against all other samples in the cohort. The plot displays the top significantly enriched Hallmark gene sets from the Molecular Signatures Database (MSigDB).

- X-axis (NES): Normalized Enrichment Score. A positive NES indicates that the gene set is significantly enriched and predominantly upregulated in the Her2-G2 cluster compared to the other samples. A negative NES indicates significant enrichment and downregulation in the Her2-G2 cluster relative to other samples (or upregulation in the comparison group).

- Y-axis: The names of the significantly enriched Hallmark gene sets.

- Bubble Size: Represents the percentage of genes within the gene set that are part of the leading edge, contributing most significantly to the enrichment score. Larger bubbles denote that a greater proportion of the gene set's members drive the observed enrichment.

- Bubble Color: Indicates the statistical significance of the enrichment result, specifically log10(1/FDR). Higher values (yellow/green) correspond to lower False Discovery Rate (FDR) q-values, indicating higher statistical confidence in the enrichment result. Lower values (purple/blue) indicate less significance (higher FDR q-values).

Interpretation: The GSEA reveals a distinct set of biological pathways differentiating the Her2-G2 cluster from the rest of the cohort. Pathways associated with various metabolic processes are significantly positively enriched (upregulated) in Her2-G2, including HALLMARK_MTORC1_SIGNALING, HALLMARK_XENOBIOTIC_METABOLISM, HALLMARK_ADIPOGENESIS, HALLMARK_CHOLESTEROL_HOMEOSTASIS, HALLMARK_FATTY_ACID_METABOLISM, HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY, HALLMARK_OXIDATIVE_PHOSPHORYLATION, and HALLMARK_PEROXISOME. Additionally, HALLMARK_ANDROGEN_RESPONSE and some immune/inflammatory pathways like HALLMARK_INFLAMMATORY_RESPONSE, HALLMARK_COMPLEMENT, and HALLMARK_ALLOGRAFT_REJECTION show positive enrichment. In contrast, pathways strongly linked to cell cycle progression and proliferation, including HALLMARK_MITOTIC_SPINDLE, HALLMARK_E2F_TARGETS, and HALLMARK_G2M_CHECKPOINT, are significantly negatively enriched (downregulated) in the Her2-G2 cluster. Similarly, HALLMARK_ESTROGEN_RESPONSE_EARLY is also negatively enriched. This suggests that while this cluster is defined by a Her2 signature and includes G2/G3 tumors, it exhibits lower proliferative activity and estrogen signaling compared to other subtypes in the cohort, potentially driven by specific metabolic and signaling adaptations. Most of the displayed enrichments are highly statistically significant, indicated by the predominantly yellow/green bubble colors.